In [1]:
import pandas as pd
import time
from bs4 import BeautifulSoup
import requests
from playwright.async_api import async_playwright, TimeoutError as PlaywrightTimeout


In [2]:
async def get_html(url, sleep=5, retries=3):
    html = None
    for i in range(1, retries+1):
        time.sleep(sleep * i)
        try:
            async with async_playwright() as p:
                browser = await p.firefox.launch()
                page = await browser.new_page()
                await page.goto(url)
                print(await page.title())
                await page.mouse.wheel(15000,15000)
                time.sleep(2)
                html = await page.innerHTML('body')
        except PlaywrightTimeout:
            print(f"Timeout error on {url}")
            continue
        else:
            break
    return html

In [3]:
import pandas as pd
import time

def get_yahoo_leagues(beg,num):    
    '''
    Scrape yahoo leagues to determine league id, points format, and teams in league
    Returns a df
    '''
    league_id = []
    league_player_count = []
    league_scoring_format = []
    league_positions = []
    
    for i in range (beg,num+1):
        try:
            df = pd.read_html(f'https://football.fantasysports.yahoo.com/f1/{i}/settings') 
            df0 = df[0]
            df1 = df[1]
            league_id.append(i)
            league_scoring_format.append(df1[df1.iloc[:,0].str.contains('Receptions')].iloc[0,1])
            league_player_count.append(int(df0[df0.Setting=='Max Teams:']['Value'].iloc[0]))
            league_positions.append(df[1].iloc[:,0].name)
          
        except:
            pass

        time.sleep(7)

    return pd.DataFrame({'id':league_id,
                         'player_count':league_player_count,'format':league_scoring_format,
                         'positions':league_positions})

In [ ]:
#df = pd.read_html(f'https://football.fantasysports.yahoo.com/f1/1/settings')

In [ ]:
# df = get_yahoo_leagues(15440,15460)
# df

In [ ]:
#leagues.append(df)

In [ ]:
#save = pd.concat(leagues)

In [ ]:
#save.to_csv('league_ids.csv')

In [272]:
# Create a df from 2010-2022 with player stats and adp
adp = pd.read_csv('adp_all.csv')
adp = adp[adp.pos.isin(['RB','WR','QB','TE'])]

df = pd.read_csv('ffb_df.csv',index_col=0)

df.Player = df.Player.str.replace('*','').str.replace('+','').str.strip()
df.Player = df.Player.str.replace('II','').str.replace('III','')
adp.name = adp.name.str.replace(' II','').str.replace('GordonI','Gordon').str.strip()
adp = adp.reset_index(drop=True)

df = df.merge(adp,how='left',left_on=['Player','FantPos','year'],right_on=['name','pos','year'])

df = df[['Rk', 'Player', 'Tm', 'FantPos', 'Age', 'G', 'GS', 'Cmp', 'Att', 'Yds',
       'TD', 'Int', 'Att.1', 'Yds.1', 'Y/A', 'TD.1', 'Tgt', 'Rec', 'Yds.2',
       'Y/R', 'TD.2', 'Fmb', 'FL', 'TD.3', 'PPR', 'VBD', 'PosRank', 'year', 'adp']]

df.loc[[7323,7326,7327,7338,7340],'adp'] = [19,25,350,38,81] #Incomplete data for adp in 2012
df.loc[df[df.adp.isna()].index,'adp'] = 350

cols = ['rk','player', 'team', 'pos', 'age', 'g', 'gs', 'cmp', 'pass_att',
       'pass_yds', 'pass_tds', 'int', 'rush_att', 'rush_yard', 'y/a',
       'rush_tds', 'tgt', 'rec', 'rec_yards', 'y/r', 'rec_tds', 'fmb', 'fl',
       'rush_rec_tds', 'ppr_pts', 'vbd', 'pos_rank', 'year',
       'adp']
df.columns = cols

df = df.fillna(0)
df = df[df.pos!=0]

In [349]:
df.head()

,rk,player,team,pos,age,g,gs,cmp,pass_att,pass_yds,...,y/r,rec_tds,fmb,fl,rush_rec_tds,ppr_pts,vbd,pos_rank,year,adp
0,1,Jamaal Charles,KAN,RB,27,15,15,0.0,0.0,0.0,...,9.90,7.0,4.0,2.0,19,378.0,182.0,1,2013,2.0
1,2,LeSean McCoy,PHI,RB,25,16,16,0.0,0.0,0.0,...,10.37,2.0,1.0,1.0,11,330.6,152.0,2,2013,9.0
2,3,Peyton Manning,DEN,QB,37,16,16,450.0,659.0,5477.0,...,0.00,0.0,10.0,6.0,1,410.0,151.0,1,2013,40.0
3,4,Matt Forte,CHI,RB,28,16,16,0.0,0.0,0.0,...,8.03,3.0,2.0,2.0,12,337.3,137.0,3,2013,12.0
4,5,Jimmy Graham,NOR,TE,27,16,12,0.0,0.0,0.0,...,14.13,16.0,0.0,0.0,16,303.5,124.0,1,2013,14.0


In [350]:
# Add 'pos_rank' and 'success' columns
df = pd.read_csv('season1.csv',index_col=0)

seasons = range(2010,2023)
positions = ['QB','RB','WR','TE']
add = []

for s in seasons:
    for p in positions:
        test = df[(df.year==s)&(df.pos==p)].sort_values('adp').reset_index(drop=True)
        test['adp_by_pos'] = test.index+1
        
        test = test[(test.year==s)&(test.pos==p)].sort_values('ppr_pts',ascending=False).reset_index(drop=True)
        test['pos_rank'] = test.index+1
        
        add.append(test)

df = pd.concat(add).reset_index(drop=True)

# Add 'success' column
for i in range(0,len(df.index)):
    if df.loc[i,'pos'] =='QB' or df.loc[i,'pos'] =='TE':
        
        if df.loc[i, 'adp_by_pos'] <= 3:
            if df.loc[i,'pos_rank'] <= 3:
                df.loc[i,'success'] = 1
            else:
                df.loc[i, 'success'] = 0
        else:
            if df.loc[i,'pos_rank']<=12:
                df.loc[i,'success'] = 1
            else:
                df.loc[i, 'success'] = 0

    else:
        if df.loc[i, 'adp_by_pos'] <= 6:
            if df.loc[i,'pos_rank'] <= 6:
                df.loc[i,'success'] = 1
            else:
                df.loc[i, 'success'] = 0

        elif df.loc[i, 'adp_by_pos'] > 36:
            if df.loc[i,'pos_rank'] < 36:
                df.loc[i,'success'] = 1
            else:
                df.loc[i, 'success'] = 0

        else:
            if df.loc[i,'pos_rank'] <= df.loc[i,'adp_by_pos']:
                df.loc[i, 'success'] = 1
            else:
                df.loc[i, 'success'] = 0
        
# Add 'round' columns
for i in range(0,len(df.index)):
    if df.loc[i,'adp'] <= 12:
        df.loc[i, 'round'] = 1
    elif df.loc[i,'adp'] <= 24:
        df.loc[i, 'round'] = 2
    elif df.loc[i,'adp'] <= 36:
        df.loc[i, 'round'] = 3
    elif df.loc[i,'adp'] <= 48:
        df.loc[i, 'round'] = 4
    elif df.loc[i,'adp'] <= 60:
        df.loc[i, 'round'] = 5
    elif df.loc[i,'adp'] <= 72:
        df.loc[i, 'round'] = 6
    elif df.loc[i,'adp'] <= 84:
        df.loc[i, 'round'] = 7
    elif df.loc[i,'adp'] <= 96:
        df.loc[i, 'round'] = 8
    elif df.loc[i,'adp'] <= 108:
        df.loc[i, 'round'] = 9
    elif df.loc[i,'adp'] <= 120:
        df.loc[i, 'round'] = 10
    elif df.loc[i,'adp'] <= 132:
        df.loc[i, 'round'] = 11
    elif df.loc[i,'adp'] <= 144:
        df.loc[i, 'round'] = 12
    elif df.loc[i,'adp'] <= 156:
        df.loc[i, 'round'] = 13
    elif df.loc[i,'adp'] <= 168:
        df.loc[i, 'round'] = 14
    else:
        df.loc[i, 'round'] = 15

In [356]:
df = pd.read_csv('season1.csv',index_col=0)

In [357]:
df.head()

,rk,player,team,pos,age,g,gs,cmp,pass_att,pass_yds,...,fl,rush_rec_tds,ppr_pts,vbd,pos_rank,year,adp,adp_by_pos,success,round
0,12,Michael Vick,PHI,QB,30,12,12,233.0,372.0,3018.0,...,3.0,9,310.3,81.0,1,2010,350.0,54,1.0,15.0
1,17,Aaron Rodgers,GNB,QB,27,15,15,312.0,475.0,3922.0,...,1.0,4,304.5,75.0,2,2010,7.0,1,1.0,1.0
2,20,Tom Brady,NWE,QB,33,16,16,324.0,492.0,3900.0,...,1.0,1,299.0,70.0,3,2010,27.0,4,1.0,3.0
3,25,Peyton Manning,IND,QB,34,16,16,450.0,679.0,4700.0,...,1.0,0,285.8,57.0,4,2010,21.0,3,0.0,2.0
4,28,Philip Rivers,SDG,QB,29,16,16,357.0,541.0,4710.0,...,4.0,0,281.6,53.0,5,2010,48.0,7,1.0,4.0


In [343]:
df.head()

,rk,player,team,pos,age,g,gs,cmp,pass_att,pass_yds,...,y/r,rec_tds,fmb,fl,rush_rec_tds,ppr_pts,vbd,pos_rank,year,adp
0,1,Jamaal Charles,KAN,RB,27,15,15,0.0,0.0,0.0,...,9.90,7.0,4.0,2.0,19,378.0,182.0,1,2013,2.0
1,2,LeSean McCoy,PHI,RB,25,16,16,0.0,0.0,0.0,...,10.37,2.0,1.0,1.0,11,330.6,152.0,2,2013,9.0
2,3,Peyton Manning,DEN,QB,37,16,16,450.0,659.0,5477.0,...,0.00,0.0,10.0,6.0,1,410.0,151.0,1,2013,40.0
3,4,Matt Forte,CHI,RB,28,16,16,0.0,0.0,0.0,...,8.03,3.0,2.0,2.0,12,337.3,137.0,3,2013,12.0
4,5,Jimmy Graham,NOR,TE,27,16,12,0.0,0.0,0.0,...,14.13,16.0,0.0,0.0,16,303.5,124.0,1,2013,14.0


In [338]:
df[(df.year==2022)][['player','pos','rk','ppr_pts','pos_rank','adp','year']].sort_values(by='rk').head(30)

,player,pos,rk,ppr_pts,pos_rank,adp,year
5502,Patrick Mahomes,QB,1,417.4,1,50.0,2022
5503,Josh Jacobs,RB,2,328.3,1,46.0,2022
5504,Christian McCaffrey,RB,3,356.4,2,3.0,2022
5505,Derrick Henry,RB,4,302.8,3,2.0,2022
5506,Justin Jefferson,WR,5,368.7,1,6.0,2022
5507,Austin Ekeler,RB,6,372.7,4,8.0,2022
5508,Josh Allen,QB,7,395.5,2,19.0,2022
5509,Travis Kelce,TE,8,316.3,1,17.0,2022
5510,Davante Adams,WR,9,335.5,2,13.0,2022
5511,Nick Chubb,RB,10,281.4,5,7.0,2022


In [137]:
df.columns

Index(['Rk', 'Player', 'Tm', 'FantPos', 'Age', 'G', 'GS', 'Cmp', 'Att', 'Yds',
       'TD', 'Int', 'Att.1', 'Yds.1', 'Y/A', 'TD.1', 'Tgt', 'Rec', 'Yds.2',
       'Y/R', 'TD.2', 'Fmb', 'FL', 'TD.3', '2:00 PM', '2PP', 'FantPt', 'PPR',
       'DKPt', 'FDPt', 'VBD', 'PosRank', 'OvRank', '-9999', 'year', 'adp',
       'name', 'pos', 'tm'],
      dtype='object')

In [234]:
df.shape

(7916, 29)

In [235]:
df.head()

,Rk,Player,Tm,FantPos,Age,G,GS,Cmp,Att,Yds,...,Y/R,TD.2,Fmb,FL,TD.3,PPR,VBD,PosRank,year,adp
0,1,Jamaal Charles,KAN,RB,27,15,15,0.0,0.0,0.0,...,9.90,7.0,4.0,2.0,19,378.0,182.0,1,2013,2.0
1,2,LeSean McCoy,PHI,RB,25,16,16,0.0,0.0,0.0,...,10.37,2.0,1.0,1.0,11,330.6,152.0,2,2013,9.0
2,3,Peyton Manning,DEN,QB,37,16,16,450.0,659.0,5477.0,...,NaN,0.0,10.0,6.0,1,410.0,151.0,1,2013,40.0
3,4,Matt Forte,CHI,RB,28,16,16,0.0,0.0,0.0,...,8.03,3.0,2.0,2.0,12,337.3,137.0,3,2013,12.0
4,5,Jimmy Graham,NOR,TE,27,16,12,0.0,0.0,0.0,...,14.13,16.0,0.0,0.0,16,303.5,124.0,1,2013,14.0


### need 'adp_by_pos', 'success', 'round'

In [164]:
df = df[~df.pos.isna()]

In [170]:
adp[adp.year==2013]

,adp,name,pos,tm,year
0,1.0,Adrian Peterson,RB,MIN,2013.0
1,2.0,Jamaal Charles,RB,KC,2013.0
2,3.0,Doug Martin,RB,TB,2013.0
3,4.0,Arian Foster,RB,HOU,2013.0
4,5.0,Calvin Johnson,WR,DET,2013.0
...,...,...,...,...,...
167,168.0,Coby Fleener,TE,IND,2013.0
168,169.0,Brian Hartline,WR,MIA,2013.0
169,170.0,Ryan Tannehill,QB,MIA,2013.0
170,171.0,Philip Rivers,QB,LAC,2013.0


In [181]:
adp.year.unique()

array([2013., 2010., 2011., 2012., 2014., 2015., 2016., 2017., 2018.,
       2019., 2020., 2021., 2022.])

In [177]:
df[(df.avg_draft_pos_ppr.isna()) & (df.rk<50)]

,rk,player,team,pos,age,g,gs,cmp,pass_att,pass_yds,...,y/r,rec_tds,fmb,fl,rush_rec_tds,ppr_pts,vbd,pos_rank,year,avg_draft_pos_ppr
23,24,Fred Jackson,BUF,RB,32,16,6,0.0,0.0,0.0,...,8.23,1.0,3.0,0.0,10,234.7,61.0,10,2013,NaN
27,28,Julius Thomas,DEN,TE,25,14,14,0.0,0.0,0.0,...,12.12,12.0,0.0,0.0,12,215.8,58.0,3,2013,NaN
30,31,Le'Veon Bell,PIT,RB,21,13,13,0.0,0.0,0.0,...,8.87,0.0,1.0,1.0,8,216.9,46.0,14,2013,NaN
43,44,Zac Stacy,STL,RB,22,14,12,0.0,0.0,0.0,...,5.42,1.0,1.0,1.0,8,183.4,31.0,18,2013,NaN
45,46,Keenan Allen,SDG,WR,21,15,14,0.0,0.0,0.0,...,14.73,8.0,2.0,2.0,8,219.6,29.0,17,2013,NaN
47,48,Julian Edelman,NWE,WR,27,16,11,0.0,0.0,0.0,...,10.06,6.0,6.0,0.0,6,251.7,27.0,18,2013,NaN
48,49,Charles Clay,MIA,TE,24,16,15,0.0,0.0,0.0,...,11.00,6.0,0.0,0.0,7,188.4,26.0,7,2013,NaN
585,2,Le'Veon Bell,PIT,RB,22,16,16,0.0,0.0,0.0,...,10.29,3.0,0.0,0.0,11,370.5,167.0,2,2014,NaN
598,15,Odell Beckham Jr.,NYG,WR,22,12,11,0.0,1.0,0.0,...,14.34,12.0,1.0,1.0,12,295.0,82.0,5,2014,NaN
599,16,Justin Forsett,BAL,RB,29,16,14,0.0,0.0,0.0,...,5.98,0.0,1.0,0.0,8,246.9,82.0,8,2014,NaN


In [179]:
adp[adp.name=='Charles Clay']

,adp,name,pos,tm,year
805,134.0,Charles Clay,TE,MIA,2014.0


In [173]:
df.shape, adp.shape

((5529, 29), (2718, 5))

In [165]:
df.isna().sum()

rk                      0
player                  0
team                    0
pos                     0
age                     0
g                       0
gs                      0
cmp                     0
pass_att                0
pass_yds                0
pass_tds                0
int                     0
rush_att                0
rush_yard               0
y/a                  2318
rush_tds                0
tgt                     2
rec                     0
rec_yards               0
y/r                   843
rec_tds                 0
fmb                     0
fl                      0
rush_rec_tds            0
ppr_pts                14
vbd                  4798
pos_rank                0
year                    0
avg_draft_pos_ppr    3589
dtype: int64

In [155]:
df.head()

,rk,player,team,pos,age,g,gs,cmp,pass_att,pass_yds,...,y/r,rec_tds,fmb,fl,rush_rec_tds,ppr_pts,vbd,pos_rank,year,avg_draft_pos_ppr
0,1,Jamaal Charles,KAN,RB,27,15,15,0.0,0.0,0.0,...,9.90,7.0,4.0,2.0,19,378.0,182.0,1,2013,2.0
1,2,LeSean McCoy,PHI,RB,25,16,16,0.0,0.0,0.0,...,10.37,2.0,1.0,1.0,11,330.6,152.0,2,2013,9.0
2,3,Peyton Manning,DEN,QB,37,16,16,450.0,659.0,5477.0,...,NaN,0.0,10.0,6.0,1,410.0,151.0,1,2013,40.0
3,4,Matt Forte,CHI,RB,28,16,16,0.0,0.0,0.0,...,8.03,3.0,2.0,2.0,12,337.3,137.0,3,2013,12.0
4,5,Jimmy Graham,NOR,TE,27,16,12,0.0,0.0,0.0,...,14.13,16.0,0.0,0.0,16,303.5,124.0,1,2013,14.0


In [154]:
df.columns = cols

In [56]:
adp[adp.name=='Christian McCaffrey']

,adp,name,pos,tm,year
1274,24.0,Christian McCaffrey,RB,CAR,2017.0
1447,13.0,Christian McCaffrey,RB,CAR,2018.0
1634,3.0,Christian McCaffrey,RB,CAR,2019.0
1832,1.0,Christian McCaffrey,RB,CAR,2020.0
2035,1.0,Christian McCaffrey,RB,CAR,2021.0
2249,3.0,Christian McCaffrey,RB,CAR,2022.0


In [48]:
df.head()

,Rk,Player,Tm,FantPos,Age,G,GS,Cmp,Att,Yds,...,2PP,FantPt,PPR,DKPt,FDPt,VBD,PosRank,OvRank,-9999,year
0,1,DeMarco Murray,DAL,RB,26,16,16,0.0,0.0,0.0,...,NaN,294.0,351.1,362.1,322.6,173.0,1,1.0,MurrDe00,2014
1,2,Le'Veon Bell,PIT,RB,22,16,16,0.0,0.0,0.0,...,NaN,288.0,370.5,376.5,329.0,167.0,2,2.0,BellLe00,2014
2,3,Marshawn Lynch,SEA,RB,28,16,14,0.0,0.0,0.0,...,NaN,265.0,302.3,310.3,283.8,144.0,3,3.0,LyncMa00,2014
3,4,Antonio Brown,PIT,WR,26,16,16,2.0,2.0,20.0,...,NaN,258.0,386.9,391.9,322.4,136.0,1,4.0,BrowAn04,2014
4,5,Matt Forte,CHI,RB,29,16,16,0.0,0.0,0.0,...,NaN,245.0,346.6,354.6,295.6,124.0,4,5.0,FortMa00,2014


In [224]:
adp.head()

,adp,name,pos,tm,year
0,1.0,Adrian Peterson,RB,MIN,2013.0
1,2.0,Jamaal Charles,RB,KC,2013.0
2,3.0,Doug Martin,RB,TB,2013.0
3,4.0,Arian Foster,RB,HOU,2013.0
4,5.0,Calvin Johnson,WR,DET,2013.0


/var/folders/fz/0282wvb93rn0lm_p0nscw1sm0000gn/T/ipykernel_20680/2210587916.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df.Player = df.Player.str.replace('*','').str.replace('+','').str.strip()
